In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# pd.options.plotting.backend = "plotly"

# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.tree import export_graphviz
from graphviz import Source

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier, plot_importance

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import VotingClassifier, RandomForestClassifier

import time

# import pandas_profiling

Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 255 ('font.family:AppleGothic')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 256 ('font.size:12')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 370 ('axes.titlesize:     large   # fontsize of the axes title')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 376 ('axes.labelsize:     medium  # fontsize of the x any y labels')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 410 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
Duplicate key in file PosixPath('/Users/jonghae/venv-tf/lib/python3.8/site-packages/matplotlib

In [2]:
pokemon = pd.read_csv('./pokemon_df_diff.csv')
pokemon.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace= True)

In [3]:
pokemon

,#_x,First_pokemon,Second_pokemon,Winner_mark,Type 1_first,Type 2_first,HP_first,Attack_first,Defense_first,Sp. Atk_first,...,Sp. Def_second,Speed_second,Generation_second,Legendary_second,HP_diff,Attack_diff,Defense_diff,Sp. Atk_diff,Sp. Def_diff,Speed_diff
0,266,266,298,2,15,10,50,64,50,45,...,40,60,3,0,-20,-6,10,-15,10,-19
1,702,702,701,2,9,5,91,90,72,90,...,90,108,5,1,0,-39,-18,18,39,0
2,191,191,668,2,4,7,55,40,85,80,...,95,40,5,0,-20,-35,10,-45,10,0
3,237,237,683,2,6,12,40,40,40,70,...,90,48,5,0,-37,-80,-50,10,-50,-28
4,151,151,231,1,15,18,70,60,125,115,...,230,5,2,0,50,50,-105,105,-160,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,707,707,126,1,2,6,100,120,100,150,...,25,60,1,0,70,80,30,80,95,30
49996,589,589,664,1,10,12,60,85,40,30,...,40,60,5,0,25,30,0,-15,5,8
49997,303,303,368,2,17,7,60,50,100,85,...,60,90,3,0,-13,-65,40,25,10,-25
49998,109,109,89,1,3,12,40,30,50,55,...,55,45,1,0,15,-5,-20,-40,0,55


### X, y split

In [4]:
cat_cols = ['#_x', 'First_pokemon', 'Second_pokemon', 
            'Legendary_first', 'Legendary_second']
conti_cols = ['HP_first', 'Attack_first',
              'Defense_first', 'Sp. Atk_first', 'Sp. Def_first', 'Speed_first',
              'Generation_first', 
              'HP_second', 'Attack_second', 'Defense_second',
              'Sp. Atk_second', 'Sp. Def_second', 'Speed_second', 'Generation_second']

y = ['Winner_mark']

In [5]:
X = pokemon[cat_cols + conti_cols]
y = pokemon[y]

In [6]:
assert len(pokemon.columns) == len (set(pokemon.columns))

In [7]:
# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=1)

# train / validation split
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=1)

y_train.shape, y_train.shape, X_test.shape, y_test.shape

((37500, 1), (37500, 1), (12500, 19), (12500, 1))

In [8]:
y_train = np.ravel (y_train, order = "c")
y_train.shape

(37500,)

In [9]:
assert len(X_train.columns) ==len(set(X_train.columns))

### Decision tree

In [10]:
pipeline_tree = make_pipeline(StandardScaler(), DecisionTreeClassifier(),verbose=True)
pipeline_tree.steps # class 이름 소문자로 변형하여 만들어준다.

[('standardscaler', StandardScaler()),
 ('decisiontreeclassifier', DecisionTreeClassifier())]

In [11]:
%%time
pipeline_tree.fit(X_train, y_train)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing decisiontreeclassifier, total=   0.3s
CPU times: user 310 ms, sys: 10.9 ms, total: 321 ms
Wall time: 320 ms


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('decisiontreeclassifier', DecisionTreeClassifier())],
         verbose=True)

In [18]:
accuracy_score(y_train, pipeline_tree.predict(X_train)), accuracy_score(y_test, pipeline_tree.predict(X_test))

(1.0, 0.91808)

### Randomforest

In [13]:
pipeline_rf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=1),verbose=True)
pipeline_rf.steps # class 이름 소문자로 변형하여 만들어준다.

[('standardscaler', StandardScaler()),
 ('randomforestclassifier', RandomForestClassifier(random_state=1))]

In [14]:
%%time
pipeline_rf.fit(X_train, y_train)

[Pipeline] .... (step 1 of 2) Processing standardscaler, total=   0.0s
[Pipeline]  (step 2 of 2) Processing randomforestclassifier, total=   4.9s
CPU times: user 4.87 s, sys: 35.5 ms, total: 4.9 s
Wall time: 4.91 s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=1))],
         verbose=True)

In [17]:
accuracy_score(y_train, pipeline_rf.predict(X_train)), accuracy_score(y_test, pipeline_rf.predict(X_test))

(0.9999733333333334, 0.94856)